In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.io as sio
import math
from tensorflow.python.framework import ops
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics.pairwise import rbf_kernel
import warnings
warnings.filterwarnings("ignore")

tf.compat.v1.disable_eager_execution()

In [ ]:
def partition_dataset(cuprite_dataset, cnv_dataset, target_labels):
    num_spectral_bands = cuprite_dataset.shape[1]
    cnv_dataset = cnv_dataset.reshape((cnv_dataset.shape[0], num_spectral_bands, 7, 7))
    train_cuprite, test_cuprite, train_cnv, test_cnv, train_labels, test_labels = train_test_split(
        cuprite_dataset, cnv_dataset, target_labels, test_size=0.2, random_state=42
    )
    train_cnv = train_cnv.reshape((train_cnv.shape[0], -1))
    test_cnv = test_cnv.reshape((test_cnv.shape[0], -1))
    return train_cuprite, test_cuprite, train_cnv, test_cnv, train_labels, test_labels

In [ ]:
def standardize(input_data):
    max_value = np.amax(input_data)
    min_value = np.amin(input_data)
    shifted_data = input_data + abs(min_value)
    normalized_data = shifted_data / (abs(min_value) + abs(max_value))
    normalized_data = normalized_data + 1e-6
    return normalized_data


In [ ]:
def sample_mask(idx, l):
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

In [ ]:
def create_placeholders(n_x, n_x1, n_y):
    training_bool = tf.compat.v1.placeholder_with_default(True, shape=())
    inp_x = tf.compat.v1.placeholder(tf.float32, [None, n_x], name="inp_x")
    inp_x1 = tf.compat.v1.placeholder(tf.float32, [None, n_x1], name="inp_x1")
    inp_y = tf.compat.v1.placeholder(tf.float32, [None, n_y], name="inp_y")
    lap_train = tf.compat.v1.placeholder(tf.float32, [None, None], name="lap_train")
    return inp_x, inp_x1, inp_y, lap_train, training_bool

In [ ]:
def adjacency_matrix(all_x, sigma_spatial=1.0, sigma_spectral=1.0):
    n = all_x.shape[0]
    spatial_distances = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                spatial_distances[i, j] = np.linalg.norm(all_x[i, :2] - all_x[j, :2])
    spectral_distances = rbf_kernel(all_x, gamma=1.0 / (2.0 * sigma_spectral**2))
    A = np.exp(-((spatial_distances / sigma_spatial)**2 + spectral_distances) / 2.0)
    return csr_matrix(A)

In [ ]:
def randomize_data(input1, input2, labels, matrix, random_seed):
    total_samples = input1.shape[0]
    np.random.seed(random_seed)
    randomized_indices = list(np.random.permutation(total_samples))
    randomized_input1 = input1[randomized_indices, :]
    randomized_input2 = input2[randomized_indices, :]
    randomized_labels = labels[randomized_indices, :].reshape((total_samples, labels.shape[1]))
    randomized_matrix1 = matrix[randomized_indices, :].reshape((matrix.shape[0], matrix.shape[1]), order="F")
    randomized_matrix = randomized_matrix1[:, randomized_indices].reshape((matrix.shape[0], matrix.shape[1]), order="F")
    return randomized_input1, randomized_input2, randomized_labels, randomized_matrix

In [ ]:
def create_mini_batches(randomized_input1, randomized_input2, randomized_labels, randomized_matrix, mini_batch_size):
    total_samples = randomized_input1.shape[0]
    mini_batches = []
    num_of_batches = math.floor(total_samples / mini_batch_size)
    for i in range(0, num_of_batches):
        start_index = i * mini_batch_size
        end_index = start_index + mini_batch_size
        mini_batch = (randomized_input1[start_index:end_index, :], randomized_input2[start_index:end_index, :],
                      randomized_labels[start_index:end_index, :], randomized_matrix[start_index:end_index, start_index:end_index])
        mini_batches.append(mini_batch)
    return mini_batches

In [ ]:
def random_min_btchs_gcn1(input1, input2, labels, matrix, mini_batch_size, random_seed):
    randomized_input1, randomized_input2, randomized_labels, randomized_matrix = randomize_data(input1, input2, labels, matrix, random_seed)
    mini_batches = create_mini_batches(randomized_input1, randomized_input2, randomized_labels, randomized_matrix, mini_batch_size)
    mini_batch = (input1, input2, labels, matrix)
    mini_batches.append(mini_batch)
    return mini_batches

In [ ]:
def gcn_layer(inp_x, adj_matrix, weights):
    x_mid = tf.matmul(inp_x, weights)
    x_out = tf.matmul(adj_matrix, x_mid)
    return x_out

In [ ]:
def initialize_params():
    tf.compat.v1.set_random_seed(1)
    initializer = tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform", seed=1)
    zeros_init = tf.compat.v1.zeros_initializer()

    params = {
        "weight1_x": tf.compat.v1.get_variable("weight1_x", [224, 128], initializer=initializer),
        "bias1_x": tf.compat.v1.get_variable("bias1_x", [128], initializer=zeros_init),
        "jweight1_x": tf.compat.v1.get_variable("jweight1_x", [128 + 128, 128], initializer=initializer),
        "jbias1_x": tf.compat.v1.get_variable("jbias1_x", [128], initializer=zeros_init),
        "jweight2_x": tf.compat.v1.get_variable("jweight2_x", [128, 12], initializer=initializer),
        "jbias2_x": tf.compat.v1.get_variable("jbias2_x", [12], initializer=zeros_init),
        "cnv_weight1_x": tf.compat.v1.get_variable("cnv_weight1_x", [3, 3, 224, 32], initializer=initializer),
        "cnv_bias1_x": tf.compat.v1.get_variable("cnv_bias1_x", [32], initializer=zeros_init),
        "cnv_weight2_x": tf.compat.v1.get_variable("cnv_weight2_x", [3, 3, 32, 64], initializer=initializer),
        "cnv_bias2_x": tf.compat.v1.get_variable("cnv_bias2_x", [64], initializer=zeros_init),
        "cnv_weight3_x": tf.compat.v1.get_variable("cnv_weight3_x", [1, 1, 64, 128], initializer=initializer),
        "cnv_bias3_x": tf.compat.v1.get_variable("cnv_bias3_x", [128], initializer=zeros_init)
    }

    return params

In [ ]:
def apply_batch_normalization(input, is_training, momentums):
    return tf.compat.v1.layers.batch_normalization(input, momentum=momentums, training=is_training)

In [ ]:
def apply_convolution(input, filters, bias, strides=[1, 1, 1, 1], padding='SAME'):
    return tf.nn.conv2d(input, filters=filters, strides=strides, padding=padding) + bias

In [ ]:
def apply_max_pooling(input, pool_size, strides, padding='SAME'):
    return tf.compat.v1.layers.max_pooling2d(input, pool_size, strides, padding=padding)

In [ ]:
def apply_relu(input):
    return tf.nn.relu(input)

In [ ]:
def apply_gcn(input, laplacian, weight, bias):
    return gcn_layer(input, laplacian, weight) + bias

In [ ]:
def apply_matmul(input, weight, bias):
    return tf.matmul(input, weight) + bias

In [ ]:
def mynetwork(x_input, x1_input, params, laplacian, is_training, momentums=0.9):
    x1_reshaped = tf.reshape(x1_input, [-1, 7, 7, 224], name="x1")

    with tf.compat.v1.name_scope("first_layer_x"):
        batch_normalization_xz1 = apply_batch_normalization(x_input, is_training, momentums)
        gcn_xz1 = apply_gcn(batch_normalization_xz1, laplacian, params['weight1_x'], params['bias1_x'])
        batch_normalization_xz1 = apply_batch_normalization(gcn_xz1, is_training, momentums)
        x_a1 = apply_relu(batch_normalization_xz1)
        cnv_xz1 = apply_convolution(x1_reshaped, params['cnv_weight1_x'], params['cnv_bias1_x'])
        batch_normalization_cnv_xz1 = apply_batch_normalization(cnv_xz1, is_training, momentums)
        pooling_cnv_xz1 = apply_max_pooling(batch_normalization_cnv_xz1, 2, 2)
        cnv_xa1 = apply_relu(pooling_cnv_xz1)

    with tf.compat.v1.name_scope("second_layer_x"):
        cnv_xz2 = apply_convolution(cnv_xa1, params['cnv_weight2_x'], params['cnv_bias2_x'])
        batch_normalization_cnv_xz2 = apply_batch_normalization(cnv_xz2, is_training, momentums)
        pooling_cnv_xz2 = apply_max_pooling(batch_normalization_cnv_xz2, 2, 2)
        x_conv_a2 = apply_relu(pooling_cnv_xz2)

    with tf.compat.v1.name_scope("third_layer_x"):
        cnv_xz3 = apply_convolution(x_conv_a2, params['cnv_weight3_x'], params['cnv_bias3_x'])
        batch_normalization_cnv_xz3 = apply_batch_normalization(cnv_xz3, is_training, momentums)
        pooling_cnv_xz3 = apply_max_pooling(batch_normalization_cnv_xz3, 2, 2)
        cnv_xa3 = apply_relu(pooling_cnv_xz3)
        cnv_xa3_shape = cnv_xa3.get_shape().as_list()
        cnv_xz3_2d = tf.reshape(cnv_xa3, [-1, cnv_xa3_shape[1] * cnv_xa3_shape[2] * cnv_xa3_shape[3]])
        jt_encd_layer = tf.concat([x_a1, cnv_xz3_2d], 1)

    with tf.compat.v1.name_scope("joint_layer_1"):
        xzj1 = apply_matmul(jt_encd_layer, params['jweight1_x'], params['jbias1_x'])
        xzj1_bn = apply_batch_normalization(xzj1, is_training, momentums)
        xaj1 = apply_relu(xzj1_bn)

    with tf.compat.v1.name_scope("x_layer_4"):
        x_zj2 = apply_matmul(xaj1, params['jweight2_x'], params['jbias2_x'])

    l2_loss = (
        tf.nn.l2_loss(params['weight1_x']) + tf.nn.l2_loss(params['jweight1_x']) +
        tf.nn.l2_loss(params['jweight2_x']) + tf.nn.l2_loss(params['cnv_weight1_x']) +
        tf.nn.l2_loss(params['cnv_weight2_x']) + tf.nn.l2_loss(params['cnv_weight3_x'])
    )

    return x_zj2, l2_loss

In [ ]:
def calculate_cost(predicted_output, actual_output, l2_loss_value, regularization):
    actual_output = tf.squeeze(actual_output, name='actual_output')
    with tf.compat.v1.name_scope("cost_calculation"):
        cost_value = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predicted_output, labels=tf.stop_gradient(actual_output))) + regularization * l2_loss_value
    return cost_value

In [ ]:
def get_update_operations():
    with tf.compat.v1.name_scope("optimization_process"):
        update_operations = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
    return update_operations

In [ ]:
def optimize(cost_value, learning_rate_value, global_step_value, update_operations):
    with tf.control_dependencies(update_operations):
        optimizer_process = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate_value).minimize(cost_value, global_step=global_step_value)
        optimizer_process = tf.group([optimizer_process, update_operations])
    return optimizer_process

In [ ]:
def optimize_network(predicted_output, actual_output, l2_loss_value, regularization, learning_rate_value, global_step_value):
    cost_value = calculate_cost(predicted_output, actual_output, l2_loss_value, regularization)
    update_operations = get_update_operations()
    optimizer_process = optimize(cost_value, learning_rate_value, global_step_value, update_operations)
    return cost_value, optimizer_process

In [ ]:
def plot_graphs(loss_values, loss_values_dev, acc_values, acc_values_dev):
    plt.plot(np.squeeze(loss_values))
    plt.plot(np.squeeze(loss_values_dev))
    plt.ylabel('Loss')
    plt.show()

    plt.plot(np.squeeze(acc_values))
    plt.plot(np.squeeze(acc_values_dev))
    plt.ylabel('Accuracy')
    plt.show()

In [ ]:
def create_model(features, features1, labels):
    tf.compat.v1.reset_default_graph()
    tf.compat.v1.set_random_seed(1)
    placeholder_input, placeholder_input1, placeholder_output, laplace_train, training_flag = create_placeholders(features, features1, labels)
    params = initialize_params()
    with tf.compat.v1.name_scope("network"):
        output_input, l2_loss_val = mynetwork(placeholder_input, placeholder_input1, params, laplace_train, training_flag)
    return placeholder_input, placeholder_input1, placeholder_output, laplace_train, training_flag, output_input, l2_loss_val, params

In [ ]:
def create_optimizer(output_input, placeholder_output, l2_loss_val, reg_beta, alpha_start, samples, batch_size):
    step = tf.Variable(0, trainable=False)
    learning_rate = tf.compat.v1.train.exponential_decay(
        alpha_start, step, 50 * samples / batch_size, 0.5, staircase=True)
    with tf.compat.v1.name_scope("optimization"):
        loss_val, optimizer_val = optimize_network(output_input, placeholder_output, l2_loss_val, reg_beta, learning_rate, step)
    return loss_val, optimizer_val

In [ ]:
def create_metrics(output_input, placeholder_output):
    with tf.compat.v1.name_scope("metrics"):
        joint_layer_transpose = tf.transpose(output_input)
        output_transpose = tf.transpose(placeholder_output)
        correct_pred = tf.equal(tf.argmax(joint_layer_transpose), tf.argmax(output_transpose))
        accuracy_val = tf.reduce_mean(tf.cast(correct_pred, "float"))
    return accuracy_val

In [ ]:
def train_epoch(sess, optimizer_val, loss_val, accuracy_val, minibatches, placeholder_input, placeholder_input1, placeholder_output, laplace_train, training_flag):
    epoch_loss, epoch_accuracy = 0., 0.
    num_minibatches = len(minibatches)
    for minibatch in minibatches:
        (batch_input, batch_input1, batch_output, batch_laplace) = minibatch
        _, minibatch_loss, minibatch_accuracy = sess.run(
            [optimizer_val, loss_val, accuracy_val],
            feed_dict={placeholder_input: batch_input, placeholder_input1: batch_input1, placeholder_output: batch_output, laplace_train: batch_laplace, training_flag: True}
        )
        epoch_loss += minibatch_loss / num_minibatches
        epoch_accuracy += minibatch_accuracy / num_minibatches
    return epoch_loss, epoch_accuracy

In [ ]:
def train_model(train_input, test_input, train_input1, test_input1, train_output, test_output, L_train_input, L_test_input, alpha_start=0.001,
                reg_beta=0.001, total_epochs=200, batch_size=32, print_loss=True):
    random_seed = 1
    samples, features = train_input.shape
    samples, labels = train_output.shape
    samples, features1 = train_input1.shape
    loss_values, loss_values_dev, train_accuracy, validation_accuracy = [], [], [], []

    placeholder_input, placeholder_input1, placeholder_output, laplace_train, training_flag, output_input, l2_loss_val, params = create_model(features, features1, labels)
    loss_val, optimizer_val = create_optimizer(output_input, placeholder_output, l2_loss_val, reg_beta, alpha_start, samples, batch_size)
    accuracy_val = create_metrics(output_input, placeholder_output)

    init = tf.compat.v1.global_variables_initializer()

    with tf.compat.v1.Session() as sess:
        sess.run(init)

        for epoch in range(total_epochs + 1):
            random_seed = random_seed + 1
            minibatches = random_min_btchs_gcn1(train_input, train_input1, train_output, L_train_input, batch_size, random_seed)
            epoch_loss, epoch_accuracy = train_epoch(sess, optimizer_val, loss_val, accuracy_val, minibatches, placeholder_input, placeholder_input1, placeholder_output, laplace_train, training_flag)

            if print_loss and (epoch) % 50 == 0:
                features, epoch_loss_dev, epoch_accuracy_dev = sess.run(
                    [output_input, loss_val, accuracy_val],
                    feed_dict={placeholder_input: test_input, placeholder_input1: test_input1, placeholder_output: test_output, laplace_train: L_test_input, training_flag: False}
                )
                print(f"Epoch {epoch}: Train_adj_Loss: {epoch_loss}, Val_loss: {epoch_loss_dev}, Train_acc: {epoch_accuracy}, Val_acc: {epoch_accuracy_dev}")

            if print_loss and epoch % 5 == 0:
                loss_values.append(epoch_loss)
                train_accuracy.append(epoch_accuracy)
                loss_values_dev.append(epoch_loss_dev)
                validation_accuracy.append(epoch_accuracy_dev)

        plot_graphs(loss_values, loss_values_dev, train_accuracy, validation_accuracy)

        params = sess.run(params)
        return params, validation_accuracy, features

In [ ]:
def load_and_standardize_data():
    graph_data_points = sio.loadmat('./cuprite_dataset/data_points_cuprite')['data_points']
    graph_data_points = standardize(graph_data_points)

    conv_data_points = sio.loadmat('./cuprite_dataset/data_points_CNN_cuprite.mat')['data_points']
    conv_data_points = standardize(conv_data_points)

    label_data = sio.loadmat('./cuprite_dataset/labels_cuprite.mat')['labels']
    label_data = label_data.T
    class_count = 12
    one_hot_labels = tf.keras.utils.to_categorical(label_data, class_count)

    return graph_data_points, conv_data_points, one_hot_labels

In [ ]:
def partition_and_create_adjacency_matrices(graph_data_points, conv_data_points, one_hot_labels):
    train_graph_data, test_graph_data, train_conv_data, test_conv_data, train_labels, test_labels = partition_dataset(graph_data_points, conv_data_points, one_hot_labels)

    train_adj_matrix = adjacency_matrix(train_graph_data)
    train_adj_matrix = train_adj_matrix.astype(int)
    train_adj_matrix = train_adj_matrix.todense()

    test_adj_matrix = adjacency_matrix(test_graph_data)
    test_adj_matrix = test_adj_matrix.astype(int)
    test_adj_matrix = test_adj_matrix.todense()

    return train_graph_data, test_graph_data, train_conv_data, test_conv_data, train_labels, test_labels, train_adj_matrix, test_adj_matrix

In [ ]:
def train_and_predict(train_graph_data, test_graph_data, train_conv_data, test_conv_data, train_labels, test_labels, train_adj_matrix, test_adj_matrix):
    trained_params, validation_accuracy, feature_set = train_model(train_graph_data, test_graph_data, train_conv_data, test_conv_data, train_labels, test_labels, train_adj_matrix, test_adj_matrix)

    predicted_labels = np.argmax(feature_set, axis=1)
    actual_labels = np.argmax(test_labels, axis=1)

    return actual_labels, predicted_labels

In [ ]:
def print_metrics(actual_labels, predicted_labels):
    print("Accuracy: ", accuracy_score(actual_labels, predicted_labels))
    print("Precision: ", precision_score(actual_labels, predicted_labels, average="macro"))
    print("Recall: ", recall_score(actual_labels, predicted_labels, average="macro"))
    print("F1: ", f1_score(actual_labels, predicted_labels, average="macro"))
    print("Confusion Matrix: ", confusion_matrix(actual_labels, predicted_labels))
    print("Classification Report: ", classification_report(actual_labels, predicted_labels))

In [ ]:
graph_data_points, conv_data_points, one_hot_labels = load_and_standardize_data()
train_graph_data, test_graph_data, train_conv_data, test_conv_data, train_labels, test_labels, train_adj_matrix, test_adj_matrix = partition_and_create_adjacency_matrices(graph_data_points, conv_data_points, one_hot_labels)
actual_labels, predicted_labels = train_and_predict(train_graph_data, test_graph_data, train_conv_data, test_conv_data, train_labels, test_labels, train_adj_matrix, test_adj_matrix)
print_metrics(actual_labels, predicted_labels)